In [1]:
def install_dependencies():
    ! rm -rf TinySQL
    ! git clone https://github.com/withmartian/TinySQL.git
    ! cd TinySQL && git checkout amir/update_eval && pip install .

install_dependencies()

Cloning into 'TinySQL'...
remote: Enumerating objects: 2125, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (222/222), done.
remote: Total 2125 (delta 141), reused 130 (delta 99), pack-reused 1804 (from 1)
Receiving objects: 100% (2125/2125), 6.87 MiB | 23.85 MiB/s, done.
Resolving deltas: 100% (1374/1374), done.
Branch 'amir/update_eval' set up to track remote branch 'amir/update_eval' from 'origin'.
Switched to a new branch 'amir/update_eval'
Processing /home/amir/codes/TinySQL/notebooks/TinySQL
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for TinySQL: filename=tinysql-1.4-py3-none-any.whl size=69033 sha256=2e46230c4651dc58ce176e6d9a0aa1378c0ec0375a5864a79f4b4e0b53cd02cb
  Stored in directory: /tmp/pip-ephem-wheel-cache-mywf1e27/wheels/d2/cc/1e/33ca2a969810eacceda354310f69088fbb0622ae8d939ee9e8
Successfully built TinySQL
  Attempting unins

In [2]:
from TinySQL.corrupt_data import CorruptFeatureTestGenerator, DEFWHEREFIELD1
from transformers import AutoTokenizer

In [3]:
model_name = "withmartian/sql_interp_bm1_cs4_dataset_synonyms_experiment_1.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [20]:
where_generator = CorruptFeatureTestGenerator(use_where=True, tokenizer=tokenizer).get_generators()[DEFWHEREFIELD1]

In [21]:
result = where_generator()

print(result)

print("\n")
print("English: " + result.english_prompt)
print("SQL: " + result.sql_statement)
print("Corrupt SQL: " + result.corrupt_sql_statement)

CorruptibleBatchItem(command_set=1, table_name=TableName(name='items', synonym='items', use_synonym=False), table_fields=[TableField(name='type', type='CHAR', synonym='category', use_synonym=False), TableField(name='label', type='TIME', synonym='tag', use_synonym=False)], create_statement='CREATE TABLE items ( type CHAR, label TIME )', select=[SelectField(name='type', aggregate='', synonym='category', use_synonym=False), SelectField(name='label', aggregate='', synonym='tag', use_synonym=False)], order_by=[], order_by_phrase='', agg_phrases=[], english_prompt='show me the type and label from the items table where label is equal to 34', sql_statement='SELECT type, label FROM items WHERE label = 34', where_fields=[TableField(name='label', type='TIME', synonym='tag', use_synonym=False)], where_literals=['34'], join_table='', join_fields=None, join_condition=None, where=['label = 34'], feature_name='DefWhereField1', clean_token_str='label', corrupt_token_str='type', clean_tokenizer_index=61

In [22]:
context = result.create_statement
english = result.english_prompt
sql = result.sql_statement
corrupt_sql = result.corrupt_sql_statement

clean_index = result.clean_tokenizer_index

In [23]:
clean_index

6167

In [24]:
alpaca_prompt = """### Instruction: {} ### Context: {} ### Response: """

full_prompt = alpaca_prompt.format(english, context) + sql

In [25]:
inputs = tokenizer(full_prompt)['input_ids']

In [26]:
inputs[result.answer_token_index]

6167

In [27]:
tokenizer.decode([clean_index])

' label'